In [1]:
import yfinance as yf
import os
import pandas as pd
import warnings
import pymongo
import json
from datetime import datetime

pd.options.mode.chained_assignment = None
warnings.filterwarnings('ignore', category=RuntimeWarning)

# Setup MongoDB connection (local)
mongo_host = "localhost"
mongo_port = 27017
mongo_user = "admin"
mongo_password = "password"
auth_db = "admin"
client_mongo = pymongo.MongoClient(
    host=mongo_host,
    port=mongo_port,
    username=mongo_user,
    password=mongo_password,
    authSource=auth_db
)
db_mongo = client_mongo.get_database("datalake")
db_mongo.drop_collection("stockmarket_brent_usd")
db_mongo.drop_collection("stockmarket_brent_eur")
db_mongo.drop_collection("stockmarket_brent_eur_liter")

# ALL YEARS
years = ['2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024', '2025']
# ONE YEARS:
# years = ['2007']

for year in years:
    print("YEAR: ", year, " // start_date:", datetime(int(year), 1, 1), "end_date:", datetime(int(year)+1, 1, 1))

     ########## CREATE BRENT_USD COLLECTION ###################
    # Télécharger les données historiques du Brent (BZ=F)
    ticker_brent = "BZ=F"
    df_brent_usd = yf.download(ticker_brent, start=(year+"-01-01"), end=year+"-12-31", auto_adjust=False)

    df_brent_usd.columns = df_brent_usd.columns.droplevel(1)
    df_brent_usd.reset_index(inplace=True)
    df_brent_usd.index.name = None

    df_brent_usd["Close"] = df_brent_usd["Close"].round(3)
    df_brent_usd["High"] = df_brent_usd["High"].round(3)
    df_brent_usd["Low"] = df_brent_usd["Low"].round(3)
    df_brent_usd["Open"] = df_brent_usd["Open"].round(3)
    df_brent_usd = df_brent_usd.drop(columns=["Adj Close"])
    print(df_brent_usd.head())

    # save df to csv
    os.makedirs("results/inputs_csv_brent_usd", exist_ok=True)
    df_brent_usd.to_csv(f"results/inputs_csv_brent_usd/Price_Brent_Usd_"+year+".csv", index=False)
    print("local save complete for brent_usd", year)
    
    # save df to mongodb stockmarket_brent_usd collection
    collection_mongo = db_mongo.get_collection("stockmarket_brent_usd")
    collection_mongo.create_index([("Date", pymongo.ASCENDING), ("Close", pymongo.ASCENDING)])
    df_brent_usd['Date'] = pd.to_datetime(df_brent_usd['Date'], format='%Y-%m-%d')
    records = df_brent_usd.to_dict(orient="records")
    collection_mongo.insert_many(records)
    print("correctly loaded", year, "brent_usd to MongoDB")
    
    ########## CREATE BRENT_EUR COLLECTION ###################
    # get collection stockmarket_eur_usd
    collection_mongo = db_mongo.get_collection("stockmarket_eur_usd")
    cursor = collection_mongo.find(
        {
            "Date": {"$gte": datetime(int(year), 1, 1), "$lt": datetime(int(year)+1, 1, 1)} 
        },
        {
            "_id": 0, "Date": 1, "Close": 1
        }
    )
    stockmarket_eur_usd = pd.DataFrame(list(cursor))
    stockmarket_eur_usd = stockmarket_eur_usd.rename(columns={"Close": "Eur_usd"})
    stockmarket_eur_usd['Date'] = pd.to_datetime(stockmarket_eur_usd['Date'])
    print("stockmarket_eur_usd")
    print(stockmarket_eur_usd.head())
    
    df_brent_usd = pd.merge(
        df_brent_usd,
        stockmarket_eur_usd,
        on='Date',
        how='left'
    )
    print("df_brent_usd with eur_usd\n", df_brent_usd.head())
    
    df_brent_eur = pd.DataFrame()
    df_brent_eur["Date"] = df_brent_usd["Date"]
    df_brent_eur["Close"] = (df_brent_usd["Close"] / df_brent_usd["Eur_usd"]).round(3)
    df_brent_eur["High"] = (df_brent_usd["High"] / df_brent_usd["Eur_usd"]).round(3)
    df_brent_eur["Low"] = (df_brent_usd["Low"] / df_brent_usd["Eur_usd"]).round(3)
    df_brent_eur["Open"] = (df_brent_usd["Open"] / df_brent_usd["Eur_usd"]).round(3)
    df_brent_eur["Volume"] = df_brent_usd["Volume"]
    print("df_brent_eur\n", df_brent_eur.head())
    
    # save df to csv
    os.makedirs("results/inputs_csv_brent_eur", exist_ok=True)
    df_brent_eur.to_csv(f"results/inputs_csv_brent_eur/Price_Brent_Eur_"+year+".csv", index=False)
    print("local save complete for brent_eur", year)
    
    # save df to mongodb stockmarket_brent_eur collection
    collection_mongo = db_mongo.get_collection("stockmarket_brent_eur")
    collection_mongo.create_index([("Date", pymongo.ASCENDING), ("Close", pymongo.ASCENDING)])
    df_brent_eur['Date'] = pd.to_datetime(df_brent_eur['Date'], format='%Y-%m-%d')
    records = df_brent_eur.to_dict(orient="records")
    collection_mongo.insert_many(records)
    print("correctly loaded", year, "brent_eur to MongoDB")
    
    
    ########## CREATE BRENT_EUR_LITER COLLECTION ###################
    # 1 Brent Barrel contains 158,987 liters
    df_brent_eur_liter = df_brent_eur.copy()
    df_brent_eur_liter["Close"] = (df_brent_eur_liter["Close"] / 158.987).round(3)
    df_brent_eur_liter["High"] = (df_brent_eur_liter["High"] / 158.987).round(3)
    df_brent_eur_liter["Low"] = (df_brent_eur_liter["Low"] / 158.987).round(3)
    df_brent_eur_liter["Open"] = (df_brent_eur_liter["Open"] / 158.987).round(3)
    print("df_brent_eur_liter\n", df_brent_eur_liter.head())
    
    # save df to csv
    os.makedirs("results/inputs_csv_brent_eur_liter", exist_ok=True)
    df_brent_eur_liter.to_csv(f"results/inputs_csv_brent_eur_liter/Price_Brent_Eur_liter_"+year+".csv", index=False)
    print("local save complete for brent_eur_liter", year)
    
    # save df to mongodb stockmarket_brent_eur_liter collection
    collection_mongo = db_mongo.get_collection("stockmarket_brent_eur_liter")
    collection_mongo.create_index([("Date", pymongo.ASCENDING), ("Close", pymongo.ASCENDING)])
    df_brent_eur_liter['Date'] = pd.to_datetime(df_brent_eur_liter['Date'], format='%Y-%m-%d')
    records = df_brent_eur_liter.to_dict(orient="records")
    collection_mongo.insert_many(records)
    print("correctly loaded", year, "brent_eur_liter to MongoDB")

YEAR:  2007  // start_date: 2007-01-01 00:00:00 end_date: 2008-01-01 00:00:00


[*********************100%***********************]  1 of 1 completed


Price       Date  Close   High    Low   Open  Volume
0     2007-07-30  75.74  76.53  75.44  75.85    2575
1     2007-07-31  77.05  77.17  75.67  75.70    3513
2     2007-08-01  75.35  77.06  74.86  77.00    3930
3     2007-08-02  75.76  76.21  74.27  75.22    6180
4     2007-08-03  74.75  76.00  74.53  75.39    4387
local save complete for brent_usd 2007
correctly loaded 2007 brent_usd to MongoDB
stockmarket_eur_usd
        Date  Eur_usd
0 2007-01-01   1.3219
1 2007-01-02   1.3276
2 2007-01-03   1.3171
3 2007-01-04   1.3093
4 2007-01-05   1.2985
df_brent_usd with eur_usd
         Date  Close   High    Low   Open  Volume  Eur_usd
0 2007-07-30  75.74  76.53  75.44  75.85    2575   1.3712
1 2007-07-31  77.05  77.17  75.67  75.70    3513   1.3672
2 2007-08-01  75.35  77.06  74.86  77.00    3930   1.3677
3 2007-08-02  75.76  76.21  74.27  75.22    6180   1.3707
4 2007-08-03  74.75  76.00  74.53  75.39    4387   1.3787
df_brent_eur
         Date   Close    High     Low    Open  Volume
0 2007

[*********************100%***********************]  1 of 1 completed


Price       Date  Close   High    Low   Open  Volume
0     2008-01-02  97.84  98.02  94.20  94.30    1149
1     2008-01-03  97.60  98.47  97.07  97.50    1386
2     2008-01-04  96.79  97.89  96.21  97.74    1240
3     2008-01-07  94.39  97.46  93.73  96.82    1826
4     2008-01-08  95.54  96.67  94.51  95.44    1082
local save complete for brent_usd 2008
correctly loaded 2008 brent_usd to MongoDB
stockmarket_eur_usd
        Date  Eur_usd
0 2008-01-01   1.4620
1 2008-01-02   1.4717
2 2008-01-03   1.4745
3 2008-01-04   1.4755
4 2008-01-07   1.4683
df_brent_usd with eur_usd
         Date  Close   High    Low   Open  Volume  Eur_usd
0 2008-01-02  97.84  98.02  94.20  94.30    1149   1.4717
1 2008-01-03  97.60  98.47  97.07  97.50    1386   1.4745
2 2008-01-04  96.79  97.89  96.21  97.74    1240   1.4755
3 2008-01-07  94.39  97.46  93.73  96.82    1826   1.4683
4 2008-01-08  95.54  96.67  94.51  95.44    1082   1.5571
df_brent_eur
         Date   Close    High     Low    Open  Volume
0 2008

...

Price       Date  Close   High    Low   Open  Volume
0     2023-01-03  82.10  87.02  81.77  86.04   27559
1     2023-01-04  77.84  82.67  77.72  82.23   24772
2     2023-01-05  78.69  79.96  77.61  78.09   28051
3     2023-01-06  78.57  80.57  78.05  78.81   23767
4     2023-01-09  79.65  81.37  78.34  78.48   29985
local save complete for brent_usd 2023
correctly loaded 2023 brent_usd to MongoDB
stockmarket_eur_usd
        Date  Eur_usd
0 2023-01-02   1.0710
1 2023-01-03   1.0678
2 2023-01-04   1.0547
3 2023-01-05   1.0606
4 2023-01-06   1.0522
df_brent_usd with eur_usd
         Date  Close   High    Low   Open  Volume  Eur_usd
0 2023-01-03  82.10  87.02  81.77  86.04   27559   1.0678
1 2023-01-04  77.84  82.67  77.72  82.23   24772   1.0547
2 2023-01-05  78.69  79.96  77.61  78.09   28051   1.0606
3 2023-01-06  78.57  80.57  78.05  78.81   23767   1.0522
4 2023-01-09  79.65  81.37  78.34  78.48   29985   1.0656
df_brent_eur
         Date   Close    High     Low    Open  Volume
0 2023

[*********************100%***********************]  1 of 1 completed


Price       Date  Close   High    Low   Open  Volume
0     2024-01-02  75.89  79.04  75.60  77.21   28591
1     2024-01-03  78.25  78.67  74.79  75.99   32172
2     2024-01-04  77.59  79.40  76.50  78.50   33154
3     2024-01-05  78.76  79.25  77.50  77.60   30994
4     2024-01-08  76.12  78.95  75.26  78.60   32539
local save complete for brent_usd 2024
correctly loaded 2024 brent_usd to MongoDB
stockmarket_eur_usd
        Date  Eur_usd
0 2024-01-01   1.1056
1 2024-01-02   1.1039
2 2024-01-03   1.0942
3 2024-01-04   1.0928
4 2024-01-05   1.0947
df_brent_usd with eur_usd
         Date  Close   High    Low   Open  Volume  Eur_usd
0 2024-01-02  75.89  79.04  75.60  77.21   28591   1.1039
1 2024-01-03  78.25  78.67  74.79  75.99   32172   1.0942
2 2024-01-04  77.59  79.40  76.50  78.50   33154   1.0928
3 2024-01-05  78.76  79.25  77.50  77.60   30994   1.0947
4 2024-01-08  76.12  78.95  75.26  78.60   32539   1.0942
df_brent_eur
         Date   Close    High     Low    Open  Volume
0 2024

[*********************100%***********************]  1 of 1 completed

Price       Date  Close   High    Low   Open  Volume
0     2025-01-02  75.93  76.54  74.73  74.88   41474
1     2025-01-03  76.51  76.74  75.53  75.90   38822
2     2025-01-06  76.30  77.50  75.94  76.59   49229
3     2025-01-07  77.05  77.27  75.92  76.24   37188
4     2025-01-08  76.16  77.89  76.03  77.24   46650
local save complete for brent_usd 2025
correctly loaded 2025 brent_usd to MongoDB
stockmarket_eur_usd
        Date  Eur_usd
0 2025-01-02   1.0352
1 2025-01-03   1.0268
2 2025-01-06   1.0305
3 2025-01-07   1.0386
4 2025-01-08   1.0346
df_brent_usd with eur_usd
         Date  Close   High    Low   Open  Volume  Eur_usd
0 2025-01-02  75.93  76.54  74.73  74.88   41474   1.0352
1 2025-01-03  76.51  76.74  75.53  75.90   38822   1.0268
2 2025-01-06  76.30  77.50  75.94  76.59   49229   1.0305
3 2025-01-07  77.05  77.27  75.92  76.24   37188   1.0386
4 2025-01-08  76.16  77.89  76.03  77.24   46650   1.0346
df_brent_eur
         Date   Close    High     Low    Open  Volume
0 2025